# Time Series Forecasting with ARIMA model

ARIMA is the short name for Auto-Regressive Integrated Moving Average.

---
## Arguments
This variable will affect the results throughout the notebook.

In [ ]:
symbol = 'GOOGL'

n_data = 1000 # number of data to analyse (0 for all data)
n_test = 250 # number of test data (must be smaller than n_data)

---
## 1. Import libraries

In [ ]:
from datetime import timedelta  

import pandas as pd
from pandas.tseries.offsets import DateOffset

import numpy as np

import matplotlib.pyplot as plt
plt.rcParams.update({'figure.figsize':(16,8)})

from statsmodels.tsa.stattools import adfuller, acf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from pmdarima import auto_arima

import warnings
warnings.filterwarnings('ignore')

---
## 2. Load Dataset

In [ ]:
csv_path = './daily_{}.csv'.format(symbol)

dataset_df = pd.read_csv(csv_path)
dataset_df

In [ ]:
dataset_df.info()

In [ ]:
dataset_df.describe()

---
## 3. Data Cleanup & Preparation

In [ ]:
# create a clean dataset with only timestamp and close columns
data_df = pd.DataFrame(dataset_df[['timestamp', 'close']]) 
data_df["timestamp"] = pd.to_datetime(data_df["timestamp"]) # convert timestamp to datetime type
data_df

In [ ]:
# sort the date by ascending order
data_df.sort_values(by="timestamp", inplace = True)

In [ ]:
# set the timestamp as index
data_df.set_index('timestamp', drop=True, inplace=True)

In [ ]:
# find the date range of the dataset
start_dt = data_df.index.min()
end_dt = data_df.index.max()
dt_idx = pd.date_range(start_dt, end_dt)

# fill those empty date with 0
data_df = data_df.reindex(dt_idx, fill_value=0)

# fill in those missing row value with the available non zero value before it
for index, row in data_df.iterrows():
    if row['close'] == 0:
        prev_index = index - timedelta(days=1)  
        row['close'] = data_df.loc[prev_index]['close']
        
data_df

In [ ]:
# get only the last N number of day (N = n_data which is define in the arguments section above)  (0 for all data)
if n_data != 0:
    start_idx = len(data_df)-min(n_data,len(data_df))
    data_df = data_df[start_idx:]
data_df.shape

In [ ]:
data_df

In [ ]:
plt.plot(data_df['close'])
plt.xlabel("Timestamp")
plt.ylabel("Close Price")
plt.show()

In [ ]:
# split the dataset to Train and Test set
train = data_df['close'][:len(data_df)-n_test]
test = data_df['close'][len(data_df)-n_test:]

In [ ]:
plt.plot(train)
plt.plot(test)
plt.xlabel("Timestamp")
plt.ylabel("Close Price")
plt.show()

## 4. Stationary Check

ARIMA model require the time series to be stationary.  
For the time series to be stationary, it's
+ Mean must be constant
+ Variance must be constant
+ There are no seasonality

### 4.1 Visualizing the Time Series Plot

In [ ]:
plt.plot(train)
plt.xlabel("Timestamp")
plt.ylabel("Close Price")
plt.show()

From the plot above, the mean is not constant throughout time. Therefore, the time series is not stationary.

### 4.2 Augmented Dickey-Fuller (ADF)
Check if the time series is stationary using Augmented Dickey-Fuller (ADF) test.

In [ ]:
#Ho: It is non stationary
#H1: It is stationary

def adfuller_test(data):
    result=adfuller(data)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

In [ ]:
adfuller_test(train.dropna())

From the above ADF result, we can see that the time series is not stationary as the p-value is way more than the threshold value or significance level of 0.05.

---
## 5. Find ARIMA model Parameters

ARIMA model require 3 parameters p, d, & q where,

+ p is the order of the Auto Regressive (AR) term which is the number of lags of Y to be used as predictors
+ q is the order of the Moving Average (MA) term which is the number of lagged forecast errors
+ d is the number of differencing required to make the time series stationary

Below are 2 methods we could use to find the parameters:
+ ACF and PACF plots method
+ AUTO_ARIMA function

### 5.1. Finding Parameters using ACF and PACF plot methods

In [ ]:
fig = plt.figure(figsize=(18,12))

ax1 = fig.add_subplot(331)
ax1.set_title('Original Series')
ax1.plot(train)

ax2 = fig.add_subplot(334)
plot_acf(train, ax=ax2)

ax3 = fig.add_subplot(337)
plot_pacf(train, ax=ax3)

ax4 = fig.add_subplot(332)
ax4.set_title('1st Order Differencing')
ax4.plot(train.diff().dropna())

ax5 = fig.add_subplot(335)
plot_acf(train.diff().dropna(), ax=ax5)

ax6 = fig.add_subplot(338)
plot_pacf(train, ax=ax6)

ax7 = fig.add_subplot(333)
ax7.set_title('2nd Order Differencing')
ax7.plot(train.diff().diff().dropna())

ax8 = fig.add_subplot(336)
plot_acf(train.diff().diff().dropna(), ax=ax8)

ax9 = fig.add_subplot(339)
plot_pacf(train, ax=ax9)

plt.show()

Looking at the 1st row plot, we will select d = 1.  
Looking at the selected d column 2nd row plot, we will select q = 0 based on MA term.  
Looking at the selected d column 3rd row plot, we will select p = 1 based on AR term.

### 5.2. Finding Parameters using AUTO_ARIMA function

In [ ]:
auto_arima(train, trace=True, error_action='ignore', suppress_warnings=True, stepwise=True)

---

## 6. Model Training

In [ ]:
arima_order = (1, 1, 0) # p, d, q

In [ ]:
# create and fit model
model = ARIMA(train, order=arima_order) 
#modelfit = model.fit(disp = 0)
model_fit = model.fit()
model_fit.summary()

In [ ]:
# show Actual vs Fitted plot
model_fit.plot_predict(dynamic=False)  
plt.show()

In [ ]:
# Plot residual errors
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(2,1)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

---
## 7. Forecast Test Data

In [ ]:
# Forecast
fc, se, conf = model_fit.forecast(n_test, alpha=0.05)  # 95% conf

# Make as pandas series
fc_series = pd.Series(fc, index=test.index)
lower_series = pd.Series(conf[:, 0], index=test.index)
upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train, label='training')
plt.plot(test, label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

---
## 8. Evaluation

In [ ]:
# Accuracy metrics
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    acf1 = acf(fc-test)[1]                      # ACF1
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'acf1':acf1, 
            'corr':corr, 'minmax':minmax})

forecast_accuracy(fc, test.values)